# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [60]:
!cat publications.tsv

pub_date	title	venue	excerpt	paper_url	slides_url	type
2022	Mental health and wellbeing during the COVID-19 period in Australia ANU Centre for Social Research and Methods			https://apo.org.au/node/318500		report
2021	The wellbeing of Australia�s children			https://assets-us-01.kc-usercontent.com/99f113b4-e5f7-00d2-23c0-c83ca2e4cfa2/e0b64280-dd99-4237-9c69-eaebc2ff3ce7/Australian-Childrens-Wellbeing-Index-Report.pdf		report
2022	Wellbeing outcomes in Australia as lockdowns ease and cases increase�August 2022			https://apo.org.au/node/319629		report
2022	The relationship between COVID-19 policies and subjective wellbeing�August 2022 ANU Centre for Social Research and Methods			https://scholar.google.com/scholar?cluster=2747851349458022468&hl=en&oi=scholarr		report
2022	Mental health and wellbeing during the COVID-19 period in Australia			https://csrm.cass.anu.edu.au/sites/default/files/docs/2022/7/Mental_health_and_wellbeing_during_the_COVID-19_period.pdf		report
2020	Gender insights in 

## Import pandas

We are using the very handy pandas library for dataframes.

In [61]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [62]:
publications = pd.read_csv("publications.csv", header=0)

def create_id(row):
    # Extract the last name of the first author
    first_author = row['authors'].split(';')[0].split(',')[0].strip()
    
    # Find the first word in the title that isn't "the" or "a"
    significant_word = next(word for word in row['title'].split() if word.lower() not in ['the', 'a'])
    
    # Combine with the year
    return f"{first_author}_{significant_word}_{row['pub_date']}"

publications['id'] = publications.apply(create_id, axis=1)

publications


,pub_date,title,authors,venue,excerpt,paper_url,slides_url,type,url_slug,id
0,2022,Mental health and wellbeing during the COVID-1...,"Biddle, Nicholas; Gray, Matthew; Rehill, Patri...",ANU Centre for Social Research and Methods wor...,NaN,https://apo.org.au/node/318500,NaN,report,apo.org.au/node/318500,Biddle_Mental_2022
1,2021,The wellbeing of Australia’s children,"Noble, Kristy; Rehill, Patrick; Sollis, Kate; ...",Australian Research Alliance for Children and ...,NaN,https://assets-us-01.kc-usercontent.com/99f113...,NaN,report,assets-us-01.kc-usercontent.com/99f113b4-e5f7-...,Noble_wellbeing_2021
2,2022,Wellbeing outcomes in Australia as lockdowns e...,"Biddle, Nicholas; Edwards, Ben; Gray, Matthew;...",ANU Centre for Social Research and Methods wor...,NaN,https://apo.org.au/node/319629,NaN,report,apo.org.au/node/319629,Biddle_Wellbeing_2022
3,2022,The relationship between COVID-19 policies and...,"Biddle, Nicholas; Edwards, Ben; Rehill, Patrick;",ANU Centre for Social Research and Methods wor...,NaN,https://scholar.google.com/scholar?cluster=274...,NaN,report,scholar.google.com/scholar?cluster=27478513494...,Biddle_relationship_2022
4,2022,Mental health and wellbeing during the COVID-1...,"Biddle, Nicholas; Gray, Matthew; Rehill, Patri...",ANU Centre for Social Research and Methods wor...,NaN,https://csrm.cass.anu.edu.au/sites/default/fil...,NaN,report,csrm.cass.anu.edu.au/sites/default/files/docs/...,Biddle_Mental_2022
5,2020,Gender insights in the Solomon Islands: Findin...,"Fisk, Kylie; McInerney, Carol; Rehill, Patrick...",International Womens Development Agency,NaN,https://scholar.google.com/scholar?cluster=531...,NaN,report,scholar.google.com/scholar?cluster=53164050568...,Fisk_Gender_2020
6,2024,Policy learning for many outcomes of interest:...,"Rehill, Patrick; Biddle, Nicholas;",Computational Economics,This explores the use of interpretable tree mo...,https://doi.org/10.1007/s10614-024-10722-1,NaN,article,doi.org/10.1007/s10614-024-10722-1,Rehill_Policy_2024
7,2021,Guidance note: Equality Insights and individua...,"Crawford, Joanne; Fisk, Kylie; Pradela, Joanna...",International Womens Development Agency,NaN,https://scholar.google.com/scholar?cluster=539...,NaN,report,scholar.google.com/scholar?cluster=53953449836...,Crawford_Guidance_2021
8,2022,Variation in policy response to COVID-19 acros...,"Edwards, Ben; Barnes, Roy; Rehill, Patrick; El...",Oxford COVID Government Response Tracker (OxCGRT),NaN,https://www.bsg.ox.ac.uk/sites/default/files/2...,NaN,report,bsg.ox.ac.uk/sites/default/files/2022-06/BSG-W...,Edwards_Variation_2022
9,2023,Fairness Implications of Heterogeneous Treatme...,"Rehill, Patrick; Biddle, Nicholas;",arXiv preprint arXiv:2309.00805,NaN,https://arxiv.org/pdf/2309.00805,NaN,preprint,arxiv.org/pdf/2309.00805,Rehill_Fairness_2023


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [63]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [64]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.id) + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = str(item.pub_date)
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: """ + item.type
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    # md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    # md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [65]:
!ls ../_publications/

Biddle_Mental_2022.md       Rehill_Causal_2024.md
Biddle_Wellbeing_2022.md    Rehill_Distilling_2024.md
Biddle_relationship_2022.md Rehill_Fairness_2023.md
Crawford_Guidance_2021.md   Rehill_How_2024.md
Edwards_Variation_2022.md   Rehill_Policy_2024.md
Fisk_Gender_2020.md         Rehill_Transparency_2024.md
Noble_wellbeing_2021.md     Taylor_Process_2023.md


In [66]:
!cat ../_publications/Rehill_Policy_2024.md

---
title: "Policy learning for many outcomes of interest: Combining optimal policy trees with multi-objective bayesian optimisation"
collection: article
permalink: /publication/2024-doi.org/10.1007/s10614-024-10722-1
excerpt: 'This explores the use of interpretable tree models with multi-objective Bayesian optimisation to explore policy eligibility trade-offs.'
date: 2024
venue: 'Computational Economics'
paperurl: 'https://doi.org/10.1007/s10614-024-10722-1'
---
This explores the use of interpretable tree models with multi-objective Bayesian optimisation to explore policy eligibility trade-offs.

[Download paper here](https://doi.org/10.1007/s10614-024-10722-1)
